# Marketing Performance & ROI Analysis

This notebook evaluates marketing performance across multiple channels and campaigns to support ROI-led budget allocation decisions.
It prioritises commercial outcomes (revenue, ROI, CPA) rather than engagement metrics in isolation.

## Business question

Which marketing channels and campaigns deliver meaningful business value, and how should budget be reallocated to improve overall performance?

## Data

This notebook uses `marketing_campaign_data.csv` located in the `data/` folder.


## Approach

- Aggregate performance by channel to compare spend, revenue contribution, CPA, and ROI.
- Drill down to campaign level to surface variance within channels.
- Produce a single stakeholder-facing snapshot visual for decision-making.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/marketing_campaign_data.csv", parse_dates=["date"])
df.head()

In [ ]:
# Quick health check
df.isna().sum()

In [ ]:
# Aggregate to channel level
ch = df.groupby("channel", as_index=False)[["spend","revenue","conversions","clicks"]].sum()
ch["cpa"] = ch["spend"] / ch["conversions"].replace(0, np.nan)
ch["roi"] = (ch["revenue"] - ch["spend"]) / ch["spend"]
ch.sort_values("roi", ascending=False)

In [ ]:
# Campaign-level view to show variance within channels
camp = df.groupby(["channel","campaign"], as_index=False)[["spend","revenue","conversions","clicks"]].sum()
camp["roi"] = (camp["revenue"] - camp["spend"]) / camp["spend"]
camp["cpa"] = camp["spend"] / camp["conversions"].replace(0, np.nan)

# Top and bottom campaigns by ROI (min spend filter to avoid noise)
camp_filtered = camp[camp["spend"] >= camp["spend"].quantile(0.5)].copy()
top = camp_filtered.sort_values("roi", ascending=False).head(10)
bottom = camp_filtered.sort_values("roi", ascending=True).head(10)
top, bottom

In [ ]:
import matplotlib.pyplot as plt

# Visual: ROI by channel (stakeholder snapshot)
ch_plot = ch.sort_values("roi", ascending=False)

plt.figure(figsize=(9,5))
plt.bar(ch_plot["channel"], ch_plot["roi"])
plt.xticks(rotation=25, ha="right")
plt.ylabel("ROI ( (Revenue - Spend) / Spend )")
plt.title("ROI by Channel (90-day view)")
plt.tight_layout()

# Save to visuals folder for the README
plt.savefig("../visuals/insight_snapshot.png", dpi=200)
plt.show()

In [ ]:
# Key findings (brief, evidence-led)
summary = ch.sort_values("roi", ascending=False)[["channel","spend","revenue","conversions","cpa","roi"]]
summary